# Notebook de pruebas PFG CDIA

## Pruebas de NER

In [ ]:
from transformers import AutoModelForTokenClassification, AutoTokenizer
import torch

In [ ]:
model_name = "ICB-UMA/ClinLinker-KB-P"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForTokenClassification.from_pretrained(model_name)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

nlp_pipeline = pipeline("ner", model=model, tokenizer=tokenizer, aggregation_strategy="simple", device=0 if torch.cuda.is_available() else -1)


In [ ]:
text = "El paciente presenta síntomas de diabetes mellitus tipo 2."

# Obtener las entidades médicas detectadas
entities = nlp_pipeline(text)

# Mostrar resultados
for entity in entities:
    print(f"Texto: {entity['word']}, Etiqueta: {entity['entity_group']}, Puntaje: {entity['score']:.2f}")


In [ ]:
from transformers import AutoTokenizer, AutoModelForMaskedLM
tokenizer = AutoTokenizer.from_pretrained("BSC-TeMU/roberta-base-biomedical-es")
model = AutoModelForMaskedLM.from_pretrained("BSC-TeMU/roberta-base-biomedical-es")
from transformers import pipeline
unmasker = pipeline('fill-mask', model="BSC-TeMU/roberta-base-biomedical-es")


In [ ]:
unmasker("El único antecedente personal a reseñar era la <mask> arterial.")

In [ ]:
unmasker("El paciente presenta síntomas de <mask> mellitus tipo 2.")

### Mejores resultados
https://huggingface.co/Helios9/BioMed_NER

In [ ]:
def merge_consecutive_entities(entities, text):
    entities = sorted(entities, key=lambda x: x['start'])
    merged_entities = []
    current_entity = None

    for entity in entities:
        if current_entity is None:
            current_entity = entity
        elif (
            entity['entity_group'] == current_entity['entity_group'] and
            (entity['start'] <= current_entity['end'])
        ):
            # Merge based on start and end positions in the text
            current_entity['end'] = max(current_entity['end'], entity['end'])
            current_entity['word'] = text[current_entity['start']:current_entity['end']]
            current_entity['score'] = (current_entity['score'] + entity['score']) / 2  
        else:
            merged_entities.append(current_entity)
            current_entity = entity
    if current_entity:
        merged_entities.append(current_entity)

    return merged_entities


In [ ]:
from transformers import pipeline

# Load the model
model_path = "Helios9/BIOMed_NER"
pipe = pipeline(
    task="token-classification",
    model=model_path,
    tokenizer=model_path,
    aggregation_strategy="simple"
)



In [ ]:
text = ("patients diagnosed in the inner lower quadrant of breast that went under lumpeoctomy")
result = pipe(text)
final_result=merge_consecutive_entities(result,text)
print(final_result)

Pruebas en castellano

In [ ]:
from transformers import pipeline
ner_pipeline = pipeline("ner", model="lcampillos/roberta-es-clinical-trials-ner", aggregation_strategy="simple")
texto = "Pacientes femeninas diagnosticadas con carcinoma adenoescamoso de pulmón."
entidades = ner_pipeline(texto)
print(entidades)

In [ ]:
final_result=merge_consecutive_entities(entidades,texto)
print(final_result)

## Búsqueda códigos para térmnos médicos

### Primera aproximación usando Snowstorm

In [ ]:
from urllib.request import urlopen, Request
from urllib.parse import quote
import json

baseUrl = 'https://snowstorm-training.snomedtools.org/snowstorm/snomed-ct'
edition = 'MAIN'

# IMPORTANT! You must update this user agent to avoid having your IP banned for 24 hours.
# Replace with a contact email so that we can contact you if your script causes excessive load on the public server
# For example: user_agent = 'example@example.com'
user_agent = 'unaiigartua2@gmail.com'

In [ ]:
def urlopen_with_header(url):
    # adds User-Agent header otherwise urlopen on its own gets an IP blocked response
    req = Request(url)
    req.add_header('User-Agent', user_agent)
    return urlopen(req)


    
 #Prints snomed code for searched disease or symptom
def getSnomedCodeSimilar(searchTerm):
    #url = baseUrl + '/browser/' + edition + '/descriptions?term=' + quote(searchTerm) + '&conceptActive=true&groupByConcept=false&searchMode=STANDARD&offset=0&limit=50'
    url = 'https://snowstorm-training.snomedtools.org/snowstorm/snomed-ct/browser/MAIN/descriptions?term=' + quote(searchTerm) + '&active=true&conceptActive=true&groupByConcept=false&searchMode=STANDARD&offset=0&limit=50'
    print(url)
    response = urlopen_with_header(url).read()
    data = json.loads(response.decode('utf-8'))

    # for term in data['items']:
    #   print(term)  
    #   if searchTerm in term['term']:
    #     print("{} : {}".format(term['term'], term['concept']['conceptId']))


    diccionario = []
    for term in data['items']:
        print(term)
        diccionario.append([term['term'], term['concept']['conceptId']])
    
    return diccionario

    

 

In [ ]:
terms = getSnomedCodeSimilar('lumpectomy')

In [ ]:
print(terms)

In [ ]:
from transformers import AutoTokenizer, AutoModel
import torch
from sklearn.metrics.pairwise import cosine_similarity


model_name = "dmis-lab/biobert-v1.1"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)


def get_mean_embedding(text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True)
    with torch.no_grad():
        outputs = model(**inputs)
    embeddings = outputs.last_hidden_state
    attention_mask = inputs["attention_mask"]
    mask_expanded = attention_mask.unsqueeze(-1).expand(embeddings.size()).float()
    masked_embeddings = embeddings * mask_expanded
    summed = torch.sum(masked_embeddings, dim=1)
    counts = torch.clamp(mask_expanded.sum(dim=1), min=1e-9)
    mean_pooled = summed / counts
    return mean_pooled



In [ ]:
# Ejemplo de textos
text1 = "inner lower quadrant of breast"
text2 = "Entire lower inner quadrant of breast"

emb1 = get_mean_embedding(text1)
emb2 = get_mean_embedding(text2)

similarity = cosine_similarity(emb1.numpy(), emb2.numpy())
print(f"Cosine Similarity: {similarity[0][0]}")


In [ ]:
emb1 = get_mean_embedding("lumpectomy")

for term in terms:
    emb2 = get_mean_embedding(term[0])
    similarity = cosine_similarity(emb1.numpy(), emb2.numpy())
    # añadir la similaridad al diccionario
    term.append(similarity[0][0])

print(terms)
    

In [ ]:
order_terms = sorted(terms, key=lambda x: x[2], reverse=True)
print(order_terms)


In [ ]:
import matplotlib.pyplot as plt

In [ ]:
nombres = [item[0] for item in order_terms]
valores = [item[2] for item in order_terms]

plt.figure(figsize=(12, 6))
plt.plot(valores, marker='o', linestyle='-')
plt.xticks(range(len(nombres)), nombres, rotation=90)
plt.xlabel("Términos")
plt.ylabel("Similaridad")
plt.title("Evolución de la Similaridad")
plt.tight_layout()
plt.show()

El problema con lo anterior es que luego hay que pasar el código de SNOMED a OMOP. No hay ninguna api que lo permita de momento.

### Segunda aproximación
Crear unos índices con todos los términos y sinónimos SNOMED y su equivalente a OMOP

Primero hacer una base de datos de los códigos de Athena con la respectiva información de cada término

In [ ]:
import pandas as pd
import sqlite3

In [ ]:
# Cargar solo las columnas necesarias
df = pd.read_csv("CONCEPT.csv", sep='\t', usecols=[
    "concept_id", "concept_name", "domain_id", "vocabulary_id",
    "concept_class_id", "standard_concept", "concept_code"
])


conn = sqlite3.connect("omop_snomed.db")
df.to_sql("concepts", conn, if_exists="replace", index=False)


conn.execute("CREATE INDEX IF NOT EXISTS idx_code ON concepts(concept_code);")
conn.execute("CREATE INDEX IF NOT EXISTS idx_name ON concepts(concept_name);")
conn.commit()


In [ ]:
def find_concept_by_code(code: str, conn):
    query = """
    SELECT concept_id, concept_name, domain_id, vocabulary_id, standard_concept
    FROM concepts
    WHERE concept_code = ?
    """
    return pd.read_sql_query(query, conn, params=(code,))


In [ ]:
# Conexión a la base
conn = sqlite3.connect("omop_snomed.db")

# Buscar por código SNOMED
find_concept_by_code("392021009", conn)

Segundo, los índices para buscar por similaridad en los sinónimos de SNOMED

In [ ]:
import pandas as pd
import numpy as np
import faiss
import pickle
from sentence_transformers import SentenceTransformer
from typing import List, Tuple


CSV_PATH = "CONCEPT_SYNONYM.csv"
MODEL_NAME = "pritamdeka/BioBERT-mnli-snli-scinli-scitail-mednli-stsb"


FAISS_INDEX_PATH = "faiss_snomed.index"
ID_MAPPING_PATH = "concept_ids.pkl"
SYNONYMS_PATH = "synonyms.parquet"



def build_vector_index():
    df = pd.read_csv(CSV_PATH, sep="\t", usecols=["concept_id", "concept_synonym_name"])
    df.drop_duplicates(inplace=True)
    df.to_parquet(SYNONYMS_PATH)

    model = SentenceTransformer(MODEL_NAME)
    texts = df["concept_synonym_name"].tolist()
    embeddings = model.encode(texts, show_progress_bar=True)
    embeddings = np.array(embeddings).astype("float32")

    index = faiss.IndexFlatL2(embeddings.shape[1])
    index.add(embeddings)

    faiss.write_index(index, FAISS_INDEX_PATH)
    with open(ID_MAPPING_PATH, "wb") as f:
        pickle.dump(df["concept_id"].tolist(), f)


def load_vector_index():
    index = faiss.read_index(FAISS_INDEX_PATH)
    with open(ID_MAPPING_PATH, "rb") as f:
        concept_ids = pickle.load(f)
    syn_df = pd.read_parquet(SYNONYMS_PATH)
    return index, concept_ids, syn_df


def search_synonym(text: str, k: int = 10) -> List[Tuple[int, str]]:
    model = SentenceTransformer(MODEL_NAME)
    query_vec = model.encode([text]).astype("float32")

    index, concept_ids, syn_df = load_vector_index()
    distances, indices = index.search(query_vec, k)

    results = []
    for idx in indices[0]:
        cid = concept_ids[idx]
        synonym = syn_df.iloc[idx]["concept_synonym_name"]
        results.append((cid, synonym))

    return results





In [ ]:
## NO EJECUTAR ESTO

build_vector_index()

In [ ]:
results = search_synonym("inner lower quadrant of breast", k=15)
for cid, name in results:
    print(f" - {cid} → {name}")

Juntar las dos partes anteriores

In [ ]:
import sqlite3

In [ ]:
def find_concept_by_code_OMOP(code: str, conn):
    query = """
    SELECT concept_id, concept_name, domain_id, vocabulary_id, standard_concept
    FROM concepts
    WHERE concept_id = ?
    """
    return pd.read_sql_query(query, conn, params=(code,))

In [ ]:
conn = sqlite3.connect("omop_snomed.db")


In [ ]:
find_concept_by_code_OMOP("4078061", conn)

In [ ]:
for cid, name in results:
    concept_info = find_concept_by_code_OMOP(cid, conn)
    print(f"Concept ID: {cid}, Concept Name: {name}, Preferred Name: {concept_info['concept_name'].values[0]}, Domain: {concept_info['domain_id'].values[0]}")

## Implementación de un RAG para obtener preguntas similares y sus correspondientes SQL

In [ ]:
# Script ragTest.py (pendiente pasarlo al notebook)

Ejemplo de ejecución:

python ragTest.py "patients diagnosed in the inner lower quadrant of breast that went under lumpeoctomy"

[•] Query: patients diagnosed in the inner lower quadrant of breast that went under lumpeoctomy

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

#1 – score 0.357
Q: How many women of reproductive age are taking estradiol? (by age group and year)

SQL (truncated): WITH drug_women AS (
    SELECT 
        p.person_id,
        YEAR(de.drug_exposure_start_date) AS year,
        YEAR(de.drug_exposure_start_date) - p.year_of_birth as age
    FROM 
        PERSON p
 …

―――――――――――――――――――――――――――――――――――――――――――――――――――――――


Full SQL for best match:

WITH drug_women AS (
    SELECT 
        p.person_id,
        YEAR(de.drug_exposure_start_date) AS year,
        YEAR(de.drug_exposure_start_date) - p.year_of_birth as age
    FROM 
        PERSON p
    JOIN 
        DRUG_EXPOSURE de ON p.person_id = de.person_id
    WHERE 
        p.gender_concept_id = 8532 AND
        de.drug_concept_id IN (1548195,46287661,19093304,1548734,19109764,1548736,1548615,35603407,1548616,35603416,1548735,19084035,46287654,35603378,19082846,1548739,19121177,1548678,1548805,19081205,19086247,1356309,40169035,1548673,1559873,1548672,1548681,1356299,1548619,19121175,19109767,1548704,1548617,19110010,1548702,1548737,19100534,19103062,1548971,19117759,40181754,19087362,19043252,40181757,1548679)
),
age_groups AS (
    SELECT 
        person_id,
        year,
        CASE
            WHEN age BETWEEN 15 AND 20 THEN '15-20'
            WHEN age BETWEEN 21 AND 24 THEN '21-24'
            WHEN age BETWEEN 25 AND 34 THEN '25-34'
            WHEN age BETWEEN 35 AND 44 THEN '35-44'
            ELSE 'Other'
        END AS age_group
    FROM 
        drug_women
)
SELECT 
    year,
    age_group,
    COUNT(DISTINCT person_id) AS patient_count
FROM 
    age_groups
GROUP BY 
    year,
    age_group
ORDER BY 
    year,
    age_group
LIMIT 10000;


━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

#2 – score 0.340
Q: what are patients counts for sleeplessness while having menopause?

SQL (truncated): WITH menopause_patients AS (
    SELECT 
        p.person_id,
        co.condition_start_date as menopause_start_date
    FROM 
        PERSON p
    JOIN 
        CONDITION_OCCURRENCE co ON p.person_i…

―――――――――――――――――――――――――――――――――――――――――――――――――――――――

#3 – score 0.314
Q: What is the number of Afib patients getting the electric Cardioversion ? Break down by year (2000-2022)

SQL (truncated): WITH afib_patients AS (
    SELECT DISTINCT person_id
    FROM condition_occurrence
    WHERE condition_concept_id IN (313217,37395821,4119602,4119601,4064452,4139517,4154290,4108832,36713962,37394031…

―――――――――――――――――――――――――――――――――――――――――――――――――――――――

#4 – score 0.312
Q: What is the 5-year bucket age distribution stratified by gender? Please calculate the age at time of first ICD10-CM H35.81 diagnosis

SQL (truncated): WITH icd10cm_h3581 AS (
    SELECT
        DISTINCT person_id,
        condition_start_date
    FROM
        condition_occurrence
    JOIN
        concept
    ON
        condition_occurrence.condition…

―――――――――――――――――――――――――――――――――――――――――――――――――――――――

#5 – score 0.306
Q: For patients with ICD10-CM have H35.81 diagnosis, what are the fifty most common treatments/therapies (at any time point)?

SQL (truncated): WITH icd10cm_h3581 AS (
    SELECT
        DISTINCT person_id
    FROM
        condition_occurrence
    JOIN
        concept
    ON
        condition_occurrence.condition_source_concept_id = concept.c…

―――――――――――――――――――――――――――――――――――――――――――――――――――――――

## Generación de las sentencias SQL

In [ ]:
# Import necessary libraries
from unsloth import FastLanguageModel
import torch
from transformers import TextStreamer


# Define the model name and other parameters
model_name = "imsanjoykb/sqlCoder-Qwen2.5-8bit"
max_seq_length = 2048
dtype = None
load_in_4bit = True

# Load the model and tokenizer from Hugging Face
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name=model_name,
    max_seq_length=max_seq_length,
    dtype=dtype,
    load_in_4bit=load_in_4bit,
)

# Enable faster inference
FastLanguageModel.for_inference(model)






In [ ]:
# load txt
db_schema = open("omop_schema_stub.txt", "r").read()

OMOP_PROMPT = """You are an expert epidemiology data analyst.
Translate the following natural-language question into a **valid SQL query**
for a database using the OMOP Common Data Model v5.3.

### OMOP schema
{db_schema}

### Question
{question}

### Similar questions
Use the following examples to help you costruct the SQL query requested.
{similar_q1}
{similar_sql1}

### SQL for {question}
"""



In [ ]:
similar_q1 = "How many patients in age between 18 and 35 or in age between 40 and 75 have anemia?"
similar_sql1 = """SELECT COUNT(DISTINCT p.person_id) AS num_patients
FROM PERSON p
JOIN CONDITION_ERA ce ON p.person_id = ce.person_id
WHERE ((YEAR(ce.condition_era_start_date) - p.year_of_birth) BETWEEN 18 AND 35
       OR (YEAR(ce.condition_era_end_date) - p.year_of_birth) BETWEEN 18 AND 35
       OR (YEAR(ce.condition_era_start_date) - p.year_of_birth) BETWEEN 40 AND 75
       OR (YEAR(ce.condition_era_end_date) - p.year_of_birth) BETWEEN 40 AND 75)
  AND ce.condition_concept_id IN ([condition@anemia])"""

In [ ]:
question = "How many patients younger than 20 or older than 40 suffered from hypertension?"
prompt = OMOP_PROMPT.format(db_schema=db_schema, question=question, similar_q1=similar_q1, similar_sql1=similar_sql1)
inputs = tokenizer([prompt], return_tensors="pt").to(model.device)

In [ ]:
streamer = TextStreamer(tokenizer, skip_prompt=True, skip_special_tokens=True)

generated = model.generate(
    **inputs,
    streamer        = streamer,
    max_new_tokens  = 500,
    temperature     = 0.1,
    top_p           = 0.95,
)
